# How to use: Backtest and Optimiser!
1. This script backtests any single sided (one way) position strategy and returns a dataframe of performance for multiple itterations (e.g different input params for signals or tp and sl targets) so as to optimise your strategy.
2. The script uses a csv from Binance spot BTC/USDT, you can use any pair provided the csv file complies with the folling criteria: columns should contain the following values and in this order [unix timestamp, open, high, low, close, volume] in the first six columns for the "READING CLEANING + SET INDEX" cell to compile correctly.
3. When testing/optimising your strategy, it is a good idea to mask the data to a smaller sample set to confirm accuracy before completing a full backtest. As a rough example to optimise 5 timeframes, with two moving averages, and four fixed TP and SL options i.e 740 iterations of Backtest() could take over 8 hours!
4. NB when optimising, should your iteration return a null or no results and therefore no values being appended to a dataframe, the optimising loop will fail. [Usually this happens if when your itteration does not return any buysignals] Hope to add an error handling fix shortly :)!

In [3]:
#Libraries
import pandas as pd
import pandas_ta as ta
import numpy as np
import itertools
#import plotly.graph_objects as go

In [4]:
#READING AND CLEANING + SET INDEX
df = pd.read_csv("../Binance_BTCUSDT_1min.csv")
df = df.iloc[:,:6]
df.columns=['Date','Open', 'High', 'Low', 'Close', 'Volume']
df.reset_index(drop=True, inplace=True)
df.Date = pd.to_datetime(df.Date)
df = df.set_index("Date")
df

,Open,High,Low,Close,Volume
Date,,,,,
2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183
2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000
2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074
2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008
2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796
...,...,...,...,...,...
2023-06-13 16:46:00,25848.59,25852.45,25843.93,25850.01,87.557760
2023-06-13 16:47:00,25850.00,25866.98,25850.00,25866.97,50.890100
2023-06-13 16:48:00,25866.98,25866.98,25855.67,25863.11,53.087330


In [5]:
#MASKING TO DECREASE SAMPLE SIZE
start_date = "2015-01-01 00:00:00"
end_date = "2024-01-01 00:00:00"
mask = (df.index > start_date) & (df.index <= end_date)
df=df.loc[mask]
df

,Open,High,Low,Close,Volume
Date,,,,,
2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183
2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000
2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074
2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008
2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796
...,...,...,...,...,...
2023-06-13 16:46:00,25848.59,25852.45,25843.93,25850.01,87.557760
2023-06-13 16:47:00,25850.00,25866.98,25850.00,25866.97,50.890100
2023-06-13 16:48:00,25866.98,25866.98,25855.67,25863.11,53.087330


In [6]:
#Function for resampling the dataframe
def resample_df(df, freq):
    resampled_open = df.Open.resample(freq).first()
    resampled_high = df.High.resample(freq).max()
    resampled_low = df.Low.resample(freq).min()
    resampled_close = df.Close.resample(freq).last()
    resampled_volume = df.Volume.resample(freq).sum()
    new_df = pd.concat([resampled_open, resampled_high, resampled_low, resampled_close, resampled_volume], axis=1)
    new_df.dropna(inplace=True)
    return new_df

In [7]:
def calc_macd(new_df, fast, slow, signal):
    macd_df = ta.macd(new_df.Close, fast, slow, signal)
    macd_df.columns=['macd', 'macd_histogram', 'macd_signal']
    new_df = pd.merge(new_df, macd_df, on='Date', how='inner')
    return new_df

In [8]:
def calc_price(new_df):
    new_df["price"] = new_df.Open.shift(-1)

In [9]:
def calc_buy_signal(new_df):
    new_df["buy_signal"] = np.where((new_df.macd>new_df.macd_signal) & (new_df.macd.shift(1)<new_df.macd_signal.shift(1)), True, False)

In [12]:
def backtest(df, freq, fast, slow, signal, tp, sl):

    new_df = resample_df(df, freq)
    new_df = calc_macd(new_df, fast, slow, signal)
    calc_price(new_df)
    calc_buy_signal(new_df)
    new_df.dropna(inplace=True)
    
    in_position = False
    trades = []
    current_trade = {}
    
    for i in range(len(new_df)-1):
        #Check exit conditions
        if in_position:
            if new_df.iloc[i].Low < current_trade["sl_price"]:
                current_trade["exit_price"] = current_trade["sl_price"]
                trades.append({
                    "entry_time":current_trade["entry_time"],
                    "entry_price":current_trade["entry_price"],
                    "tp_target":current_trade["tp_price"],
                    "sl_target":current_trade["sl_price"],
                    "exit_time":new_df.iloc[i].name,
                    "exit_price":current_trade["exit_price"],
                    "pnl":(current_trade["exit_price"]-current_trade["entry_price"])/current_trade["exit_price"],
                })
                current_trade = {}
                in_position = False
                
            elif new_df.iloc[i].High > current_trade["tp_price"]:
                current_trade["exit_price"] = current_trade["tp_price"]
                trades.append({
                    "entry_time":current_trade["entry_time"],
                    "entry_price":current_trade["entry_price"],
                    "tp_target":current_trade["tp_price"],
                    "sl_target":current_trade["sl_price"],
                    "exit_time":new_df.iloc[i].name,
                    "exit_price":current_trade["tp_price"],
                    "pnl":(current_trade["exit_price"]/current_trade["entry_price"])-1,
                })
                current_trade = {}
                in_position = False
        
        #Enter the trade
        if not in_position:
            if new_df.iloc[i].buy_signal == True:
                current_trade["entry_price"] = new_df.iloc[i].price
                current_trade["entry_time"] = new_df.iloc[i+1].name
                current_trade["tp_price"] = new_df.iloc[i].price*tp
                current_trade["sl_price"] = new_df.iloc[i].price*sl
                in_position = True
                
    data = pd.DataFrame(trades)
    amount = len(data)
    winrate = len(data.loc[data.pnl.values>0])/len(data)*100
    pnl = sum(pd.Series(data.pnl))
    return amount, winrate, pnl
    #return pd.DataFrame(trades) 

In [13]:
#def backtest(df, freq, fast, slow, signal, tp, sl):
backtest(df, "4H", 12, 26, 9, 1.02, 0.97)

(424, 60.84905660377359, 0.02597938144329878)

# Optimiser

In [17]:
# Define the parameter combinations
freq_values = ["1H", "4H", "1D"]
macd_fast_values = [12]
macd_slow_values = [26]
macd_signal_values = [9, 21]
tp_values = [1.02, 1.03, 1.04, 1.12]
sl_values = [0.98, 0.97, 0.96]

In [19]:
#Create an empty dataframe
results_df = pd.DataFrame(columns=["freq", "macd_fast", "macd_slow", "macd_signal", "tp", "sl", "amount", "winrate", "pnl"])

In [20]:
for freq, macd_fast, macd_slow, macd_signal, tp, sl in itertools.product(freq_values, macd_fast_values, macd_slow_values, macd_signal_values, tp_values, sl_values):
    amount, winrate, pnl = backtest(df, freq, macd_fast, macd_slow, macd_signal, tp, sl)
    result = pd.DataFrame([[freq, macd_fast, macd_slow, macd_signal, tp, sl, amount, winrate, pnl]], columns=["freq", "macd_fast", "macd_slow", "macd_signal", "tp", "sl", "amount", "winrate", "pnl"])
    results_df = pd.concat([results_df, result], ignore_index=True)

# Print the results DataFrame
print(results_df)

   freq macd_fast macd_slow macd_signal    tp    sl amount    winrate  \
0    1H        12        26           9  1.02  0.98   1424  50.772472   
1    1H        12        26           9  1.02  0.97   1245  60.080321   
2    1H        12        26           9  1.02  0.96   1130  65.929204   
3    1H        12        26           9  1.03  0.98   1294  39.721793   
4    1H        12        26           9  1.03  0.97   1096  48.266423   
..  ...       ...       ...         ...   ...   ...    ...        ...   
67   1D        12        26          21  1.04  0.97     50  34.000000   
68   1D        12        26          21  1.04  0.96     49  48.979592   
69   1D        12        26          21  1.12  0.98     51  13.725490   
70   1D        12        26          21  1.12  0.97     50  20.000000   
71   1D        12        26          21  1.12  0.96     48  31.250000   

         pnl  
0   0.153878  
1  -0.411134  
2  -1.141667  
3  -0.498367  
4  -1.666082  
..       ...  
67 -0.340619  
68 

In [21]:
results_df

,freq,macd_fast,macd_slow,macd_signal,tp,sl,amount,winrate,pnl
0,1H,12,26,9,1.02,0.98,1424,50.772472,0.153878
1,1H,12,26,9,1.02,0.97,1245,60.080321,-0.411134
2,1H,12,26,9,1.02,0.96,1130,65.929204,-1.141667
3,1H,12,26,9,1.03,0.98,1294,39.721793,-0.498367
4,1H,12,26,9,1.03,0.97,1096,48.266423,-1.666082
...,...,...,...,...,...,...,...,...,...
67,1D,12,26,21,1.04,0.97,50,34.000000,-0.340619
68,1D,12,26,21,1.04,0.96,49,48.979592,-0.081667
69,1D,12,26,21,1.12,0.98,51,13.725490,-0.057959
70,1D,12,26,21,1.12,0.97,50,20.000000,-0.037113


In [22]:
#rounding the pnl
results_df["pnl1"] = results_df.pnl.round(2)

In [27]:
filtered_results = results_df[(results_df["winrate"] > 50) & (results_df["amount"] > 100) & (results_df["pnl1"] > 0.4) & (results_df["sl"]>0.9)]
filtered_results

,freq,macd_fast,macd_slow,macd_signal,tp,sl,amount,winrate,pnl,pnl1
29,4H,12,26,9,1.03,0.96,394,59.64467,0.425,0.43


In [ ]:
results_df.to_csv('BTCUSDT_MACDTest-Backtest.csv', index=True)

In [29]:
pd.read_csv('../BTCUSDT_RSIBB_BREAKOUT-Backtest.csv')

,Unnamed: 0,freq,rsi_length,sma_period,overSold,tp,sl,amount,winrate,pnl
0,0,1H,7,20,20,1.02,0.98,732,52.322404,0.537551
1,1,1H,7,20,20,1.02,0.97,678,62.536873,0.624330
2,2,1H,7,20,20,1.02,0.96,644,68.167702,0.238333
3,3,1H,7,20,20,1.02,0.88,456,85.526316,-1.200000
4,4,1H,7,20,20,1.03,0.98,643,43.856921,1.092653
...,...,...,...,...,...,...,...,...,...,...
859,859,8H,14,200,30,1.04,0.88,185,77.837838,0.169091
860,860,8H,14,200,30,1.12,0.98,188,17.021277,0.656327
861,861,8H,14,200,30,1.12,0.97,151,23.841060,0.763299
862,862,8H,14,200,30,1.12,0.96,134,27.611940,0.398333


# Visualisation

In [ ]:
#CANDLE STICK AND MERGED PLOTS
fig = go.Figure(data=[go.Candlestick(
    x = new_df.index,
    open = new_df.Open,
    high = new_df.High,
    low = new_df.Low,
    close = new_df.Close,
    increasing_line_color = "rgba(0, 0, 0, 0.5)",
    decreasing_line_color = "rgba(0, 0, 0, 0.5)",
    name="Candlesticks"
    ),
    go.Scatter(x=new_df.index, y=new_df.VWAP, line=dict(color='red', width=1)),
    go.Scatter(x=new_df.index, y=new_df.SMA, line=dict(color='blue', width=1))
])

#ENTRY PLOTS
fig.add_trace(go.Scatter(
    x=trades.entry_time,
    y=trades.entry_price,
    mode = "markers",
    customdata=trades,
    marker_symbol="triangle-up",
    marker_color="rgba(0, 255, 0, 0.9)",
    marker_line_color="rgba(0, 0, 0, 0.5)",
    marker_size =13,
    marker_line_width=1,
    name="Entries",
    hovertemplate="Entry Time: %{customdata[1]}<br>"
        "Entry Price: %{y:.2f}<br>"
        "Size: %{customdata[2]:.6f}"
))

#EXIT PLOTS
fig.add_trace(go.Scatter(
    x=trades.exit_time,
    y=trades.exit_price,
    mode = "markers",
    customdata=trades,
    marker_symbol="triangle-down",
    marker_color="rgba(255, 0, 0, 0.9)",
    marker_line_color="rgba(0, 0, 0, 0.5)",
    marker_size =13,
    marker_line_width=1,
    name="Exits",
    hovertemplate="Exit Time: %{customdata[4]}<br>"
        "Exit Price: %{y:.2f}<br>"
        #"Size: %{customdata[2]:.6f}"
))

#fig.update_layout (xaxis_rangeslider_visable = False)
fig